In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import cv2
import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import json
import os

2024-07-15 20:15:54.045697: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 20:16:02.874251: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
#importing data
classes_train = pd.read_csv('../data/train/_classes.csv')
classes_test = pd.read_csv('../data/test/_classes.csv')
classes_valid = pd.read_csv('../data/valid/_classes.csv')

pics_train = []
pics_valid = []
pics_test = []


for filename in classes_train['filename']:
    img = cv2.imread('../data/train/'+filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    pics_train.append(img)

for filename in classes_valid['filename']:
    img = cv2.imread('../data/valid/'+filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    pics_valid.append(img)

for filename in classes_test['filename']:
    img = cv2.imread('../data/test/'+filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    pics_test.append(img)


In [3]:
pics_train = np.array(pics_train)
pics_valid = np.array(pics_valid)
pics_test = np.array(pics_test)

In [4]:
def crop_face(image):
    """
    Crops the face from input image using Haar Cascade Classifier

    Args:
        image: image

    Returns:
        face: face image
    """
    face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(image, 1.1, 4)
    if len(faces) == 0:
        return image
    for (x, y, w, h) in faces:
        if y-30 >= 0 or x-30 >= 0 or y+h+30 < image.shape[0] or x+w+30 < image.shape[1]:
            face = cv2.resize(image[y-30:y+30+h, x-30:x+w+30], (160, 160))
        else:
            face = cv2.resize(image[y:y+h, x:x+w], (160, 160))
    return face


# faces = []
# for image in tqdm.tqdm(pics_train[:100]):
#     face = crop_face(image)
#     if face is not None:
#         faces.append(face)

# plt.imshow(pics_train[0])

# test = crop_face(pics_train[0])

# plt.imshow(test, cmap='gray')

In [5]:
def hard_crop(image):
    return image[:,:,100:image.shape[2]-100]

In [6]:
train_cropped = hard_crop(pics_train)
valid_cropped = hard_crop(pics_valid)
test_cropped = hard_crop(pics_test)

In [7]:
all_data_merged = pd.concat([classes_train, classes_test, classes_valid])
print(all_data_merged[" DangerousDriving"].sum())
print(all_data_merged[" SafeDriving"].sum())
print(all_data_merged[" Yawn"].sum())
print(all_data_merged[" Distracted"].sum())
print(all_data_merged[" Drinking"].sum())
print(all_data_merged[" SleepyDriving"].sum())

ratio_dangerous = all_data_merged[" DangerousDriving"].sum() / all_data_merged[" SafeDriving"].sum()
ratio_yawn = all_data_merged[" Yawn"].sum() / all_data_merged[" SafeDriving"].sum()
ratio_distracted = all_data_merged[" Distracted"].sum() / all_data_merged[" SafeDriving"].sum()
ratio_drinking = all_data_merged[" Drinking"].sum() / all_data_merged[" SafeDriving"].sum()
ratio_sleepy = all_data_merged[" SleepyDriving"].sum() / all_data_merged[" SafeDriving"].sum()

pics_per_pic = [1, 11, 3, 14, 6]


4642
6180
546
2080
428
979


In [8]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def augment_images(images, num_augmentations):
    """
    Augment images using ImageDataGenerator

    Args:
        images (numpy.ndarray): Images to augment
    
    Returns:
        numpy.ndarray: Augmented images
    """
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.3,
        height_shift_range=0.3,  # Increased height shift range
        shear_range=0.3,
        zoom_range=0.3,
        vertical_flip=True,
        fill_mode='nearest'
    )
    augmented_images = []
    for image in images:
        image = np.expand_dims(image, 0)
        i = 0
        for i in range(num_augmentations):
            augmented_images.append(train_datagen.random_transform(image))
            i += 1
            if i >= num_augmentations:
                break
    return augmented_images

train_yawn_augmented = []
train_yawn_data_augmented = pd.DataFrame(columns=classes_train.columns)
k = 0  # Initialize k before the loop

for i, elem in enumerate(classes_train[" Yawn"]):
    if elem == 1:
        images = augment_images([train_cropped[i]], 11)
        train_yawn_augmented.append(images)
        for j in range(11):
            train_yawn_data_augmented = pd.concat([train_yawn_data_augmented, classes_train.iloc[[i]]], ignore_index=True)
            train_yawn_data_augmented.at[k+j, "filename"] = f"{k+j}"
        k += 11  # Increment k after processing each set of images

#do same for  Distracted, Drinking and SleepyDriving
train_distracted_augmented = []
train_distracted_data_augmented = pd.DataFrame(columns=classes_train.columns)
k = 0  # Initialize k before the loop

for i, elem in enumerate(classes_train[" Distracted"]):
    if elem == 1:
        images = augment_images([train_cropped[i]], 3)
        train_distracted_augmented.append(images)
        for j in range(3):
            train_distracted_data_augmented = pd.concat([train_distracted_data_augmented, classes_train.iloc[[i]]], ignore_index=True)
            train_distracted_data_augmented.at[k+j, "filename"] = f"{k+j}"
        k += 3  # Increment k after processing each set of images

train_drinking_augmented = []
train_drinking_data_augmented = pd.DataFrame(columns=classes_train.columns)
k = 0  # Initialize k before the loop

for i, elem in enumerate(classes_train[" Drinking"]):
    if elem == 1:
        images = augment_images([train_cropped[i]], 14)
        train_drinking_augmented.append(images)
        for j in range(14):
            train_drinking_data_augmented = pd.concat([train_drinking_data_augmented, classes_train.iloc[[i]]], ignore_index=True)
            train_drinking_data_augmented.at[k+j, "filename"] = f"{k+j}"
        k += 14  # Increment k after processing each set of images

train_sleepy_augmented = []
train_sleepy_data_augmented = pd.DataFrame(columns=classes_train.columns)
k = 0  # Initialize k before the loop

for i, elem in enumerate(classes_train[" SleepyDriving"]):
    if elem == 1:
        images = augment_images([train_cropped[i]], 6)
        train_sleepy_augmented.extend(images)
        for j in range(6):
            train_sleepy_data_augmented = pd.concat([train_sleepy_data_augmented, classes_train.iloc[[i]]], ignore_index=True)
            train_sleepy_data_augmented.at[k+j, "filename"] = f"{k+j}"
        k += 6  # Increment k after processing each set of images

test_yawn_augmented = []
test_yawn_data_augmented = pd.DataFrame(columns=classes_test.columns)
k = 0  # Initialize k before the loop

for i, elem in enumerate(classes_test[" Yawn"]):
    if elem == 1:
        images = augment_images([test_cropped[i]], 11)
        test_yawn_augmented.append(images)
        for j in range(11):
            test_yawn_data_augmented = pd.concat([test_yawn_data_augmented, classes_test.iloc[[i]]], ignore_index=True)
            test_yawn_data_augmented.at[k+j, "filename"] = f"{k+j}"
        k += 11  # Increment k after processing each set of images

#do same for  Distracted, Drinking and SleepyDriving
test_distracted_augmented = []
test_distracted_data_augmented = pd.DataFrame(columns=classes_test.columns)
k = 0  # Initialize k before the loop

for i, elem in enumerate(classes_test[" Distracted"]):
    if elem == 1:
        images = augment_images([test_cropped[i]], 3)
        test_distracted_augmented.append(images)
        for j in range(3):
            test_distracted_data_augmented = pd.concat([test_distracted_data_augmented, classes_test.iloc[[i]]], ignore_index=True)
            test_distracted_data_augmented.at[k+j, "filename"] = f"{k+j}"
        k += 3  # Increment k after processing each set of images

test_drinking_augmented = []
test_drinking_data_augmented = pd.DataFrame(columns=classes_test.columns)
k = 0  # Initialize k before the loop

for i, elem in enumerate(classes_test[" Drinking"]):
    if elem == 1:
        images = augment_images([test_cropped[i]], 14)
        test_drinking_augmented.append(images)
        for j in range(14):
            test_drinking_data_augmented = pd.concat([test_drinking_data_augmented, classes_test.iloc[[i]]], ignore_index=True)
            test_drinking_data_augmented.at[k+j, "filename"] = f"{k+j}"
        k += 14  # Increment k after processing each set of images

test_sleepy_augmented = []
test_sleepy_data_augmented = pd.DataFrame(columns=classes_test.columns)
k = 0  # Initialize k before the loop

for i, elem in enumerate(classes_test[" SleepyDriving"]):
    if elem == 1:
        images = augment_images([test_cropped[i]], 6)
        test_sleepy_augmented.extend(images)
        for j in range(6):
            test_sleepy_data_augmented = pd.concat([test_sleepy_data_augmented, classes_test.iloc[[i]]], ignore_index=True)
            test_sleepy_data_augmented.at[k+j, "filename"] = f"{k+j}"
        k += 6  # Increment k after processing each set of images

valid_yawn_augmented = []
valid_yawn_data_augmented = pd.DataFrame(columns=classes_valid.columns)
k = 0  # Initialize k before the loop

for i, elem in enumerate(classes_valid[" Yawn"]):
    if elem == 1:
        images = augment_images([valid_cropped[i]], 11)
        valid_yawn_augmented.append(images)
        for j in range(11):
            valid_yawn_data_augmented = pd.concat([valid_yawn_data_augmented, classes_valid.iloc[[i]]], ignore_index=True)
            valid_yawn_data_augmented.at[k+j, "filename"] = f"{k+j}"
        k += 11  # Increment k after processing each set of images

#do same for  Distracted, Drinking and SleepyDriving
valid_distracted_augmented = []
valid_distracted_data_augmented = pd.DataFrame(columns=classes_valid.columns)
k = 0  # Initialize k before the loop

for i, elem in enumerate(classes_valid[" Distracted"]):
    if elem == 1:
        images = augment_images([valid_cropped[i]], 3)
        valid_distracted_augmented.append(images)
        for j in range(3):
            valid_distracted_data_augmented = pd.concat([valid_distracted_data_augmented, classes_valid.iloc[[i]]], ignore_index=True)
            valid_distracted_data_augmented.at[k+j, "filename"] = f"{k+j}"
        k += 3  # Increment k after processing each set of images

valid_drinking_augmented = []
valid_drinking_data_augmented = pd.DataFrame(columns=classes_valid.columns)
k = 0  # Initialize k before the loop

for i, elem in enumerate(classes_valid[" Drinking"]):
    if elem == 1:
        images = augment_images([valid_cropped[i]], 14)
        valid_drinking_augmented.append(images)
        for j in range(14):
            valid_drinking_data_augmented = pd.concat([valid_drinking_data_augmented, classes_valid.iloc[[i]]], ignore_index=True)
            valid_drinking_data_augmented.at[k+j, "filename"] = f"{k+j}"
        k += 14  # Increment k after processing each set of images

valid_sleepy_augmented = []
valid_sleepy_data_augmented = pd.DataFrame(columns=classes_valid.columns)
k = 0  # Initialize k before the loop

for i, elem in enumerate(classes_valid[" SleepyDriving"]):
    if elem == 1:
        images = augment_images([valid_cropped[i]], 6)
        valid_sleepy_augmented.extend(images)
        for j in range(6):
            valid_sleepy_data_augmented = pd.concat([valid_sleepy_data_augmented, classes_valid.iloc[[i]]], ignore_index=True)
            valid_sleepy_data_augmented.at[k+j, "filename"] = f"{k+j}"
        k += 6  # Increment k after processing each set of images



In [9]:
train_yawn_augmented = np.array(train_yawn_augmented).reshape(-1, 360, 440)
train_distracted_augmented = np.array(train_distracted_augmented).reshape(-1, 360, 440)
train_drinking_augmented = np.array(train_drinking_augmented).reshape(-1, 360, 440)
train_sleepy_augmented = np.array(train_sleepy_augmented).reshape(-1, 360, 440)
test_yawn_augmented = np.array(test_yawn_augmented).reshape(-1, 360, 440)
test_distracted_augmented = np.array(test_distracted_augmented).reshape(-1, 360, 440)
test_drinking_augmented = np.array(test_drinking_augmented).reshape(-1, 360, 440)
test_sleepy_augmented = np.array(test_sleepy_augmented).reshape(-1, 360, 440)
valid_yawn_augmented = np.array(valid_yawn_augmented).reshape(-1, 360, 440)
valid_distracted_augmented = np.array(valid_distracted_augmented).reshape(-1, 360, 440)
valid_drinking_augmented = np.array(valid_drinking_augmented).reshape(-1, 360, 440)
valid_sleepy_augmented = np.array(valid_sleepy_augmented).reshape(-1, 360, 440)




In [10]:
train_img_aug = []
test_img_aug = []
valid_img_aug = []

for pic in tqdm.tqdm(range(len(train_yawn_augmented))):
    img = cv2.resize(train_yawn_augmented[pic], (192, 160))
    train_img_aug.append(img)

for pic in tqdm.tqdm(range(len(train_distracted_augmented))):
    img = cv2.resize(train_distracted_augmented[pic], (192, 160))
    train_img_aug.append(img)

for pic in tqdm.tqdm(range(len(train_drinking_augmented))):
    img = cv2.resize(train_drinking_augmented[pic], (192, 160))
    train_img_aug.append(img)

for pic in tqdm.tqdm(range(len(train_sleepy_augmented))):
    img = cv2.resize(train_sleepy_augmented[pic], (192, 160))
    train_img_aug.append(img)

for pic in tqdm.tqdm(range(len(test_yawn_augmented))):
    img = cv2.resize(test_yawn_augmented[pic], (192, 160))
    test_img_aug.append(img)

for pic in tqdm.tqdm(range(len(test_distracted_augmented))):
    img = cv2.resize(test_distracted_augmented[pic], (192, 160))
    test_img_aug.append(img)

for pic in tqdm.tqdm(range(len(test_drinking_augmented))):
    img = cv2.resize(test_drinking_augmented[pic], (192, 160))
    test_img_aug.append(img)

for pic in tqdm.tqdm(range(len(test_sleepy_augmented))):
    img = cv2.resize(test_sleepy_augmented[pic], (192, 160))
    test_img_aug.append(img)

for pic in tqdm.tqdm(range(len(valid_yawn_augmented))):
    img = cv2.resize(valid_yawn_augmented[pic], (192, 160))
    valid_img_aug.append(img)

for pic in tqdm.tqdm(range(len(valid_distracted_augmented))):
    img = cv2.resize(valid_distracted_augmented[pic], (192, 160))
    valid_img_aug.append(img)

for pic in tqdm.tqdm(range(len(valid_drinking_augmented))):
    img = cv2.resize(valid_drinking_augmented[pic], (192, 160))
    valid_img_aug.append(img)

for pic in tqdm.tqdm(range(len(valid_sleepy_augmented))):
    img = cv2.resize(valid_sleepy_augmented[pic], (192, 160))
    valid_img_aug.append(img)

#concat the df in the same order
train_data_aug = pd.concat([train_yawn_data_augmented, train_distracted_data_augmented, train_drinking_data_augmented, train_sleepy_data_augmented])
test_data_aug = pd.concat([test_yawn_data_augmented, test_distracted_data_augmented, test_drinking_data_augmented, test_sleepy_data_augmented])
valid_data_aug = pd.concat([valid_yawn_data_augmented, valid_distracted_data_augmented, valid_drinking_data_augmented, valid_sleepy_data_augmented])


train_img_aug = np.array(train_img_aug)
valid_img_aug = np.array(valid_img_aug)
test_img_aug = np.array(test_img_aug)

100%|██████████| 750/750 [00:00<00:00, 12174.06it/s]


In [11]:
# train_img = []
# valid_img = []
# test_img = []

# for pic in tqdm.tqdm(range(len(train_cropped))):
#     img = cv2.resize(train_cropped[pic], (192, 160))
#     train_img.append(img)

# for pic in tqdm.tqdm(range(len(valid_cropped))):
#     img = cv2.resize(valid_cropped[pic], (192, 160))
#     valid_img.append(img)

# for pic in tqdm.tqdm(range(len(test_cropped))):
#     img = cv2.resize(test_cropped[pic], (192, 160))
#     test_img.append(img)

# train_img = np.array(train_img)
# valid_img = np.array(valid_img)
# test_img = np.array(test_img)

In [12]:
# train_img_aug = train_img_aug.tolist()
# valid_img_aug = valid_img_aug.tolist()
# test_img_aug = test_img_aug.tolist()

# json_train_aug = os.path.join('../data/augmented_data', 'train.json')
# json_valid_aug = os.path.join('../data/augmented_data', 'valid.json')
# json_test_aug =  os.path.join('../data/augmented_data', 'test.json')

# # file_path = os.path.join('data', 'array.json')

# with open(json_train_aug, 'w') as f:
#     json.dump(train_img_aug, f)

# with open(json_valid_aug, 'w') as f:
#     json.dump(valid_img_aug, f)

# with open(json_test_aug, 'w') as f:
#     json.dump(test_img_aug, f)

In [13]:
#safe the augmented dataframes
train_data_aug.to_csv('../data/augmented_data/train.csv', index=False)
valid_data_aug.to_csv('../data/augmented_data/valid.csv', index=False)
test_data_aug.to_csv('../data/augmented_data/test.csv', index=False)

In [14]:
plt.imshow(test_img[34], cmap='gray')

NameError: name 'test_img' is not defined

In [ ]:


train_img = train_img.tolist()
valid_img = valid_img.tolist()
test_img = test_img.tolist()

json_train = os.path.join('../data', 'train.json')
json_valid = os.path.join('../data', 'valid.json')
json_test =  os.path.join('../data', 'test.json')

# file_path = os.path.join('data', 'array.json')

with open(json_train, 'w') as f:
    json.dump(train_img, f)

with open(json_valid, 'w') as f:
    json.dump(valid_img, f)

with open(json_test, 'w') as f:
    json.dump(test_img, f)